In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
from shapely.geometry import LineString
import pandas as pd
import time

start_time = time.time() 
toy_node_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\Korea\KOREARL_NODE.shp"
toy_edge_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\Korea\KOREARL_EDGE.shp"

toy_node = gpd.read_file(toy_node_path)
toy_edge = gpd.read_file(toy_edge_path)

toy_node = toy_node.to_crs(epsg=3857)
toy_edge = toy_edge.to_crs(epsg=3857)

fig, ax = plt.subplots(figsize=(7, 7))
toy_edge.plot(ax=ax, color="grey", linewidth=1.8, label="Edges")

node_coords = toy_node.geometry.apply(lambda pt: (pt.x, pt.y)).tolist()
x_coords, y_coords = zip(*node_coords)
ax.scatter(x_coords, y_coords, color="black", s=2.3, label="Nodes", zorder=3, alpha=0.5)

ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron, zorder=0)
plt.legend()
plt.title("Scotland Railway Network")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.tight_layout()
plt.show()

In [ ]:
import networkx as nx



# 1. Node numbering (n1, n2, ...)
round_coord = lambda coord: (round(coord[0], 6), round(coord[1], 6))

# 1. node_id → geometry
node_info_list = [
    (row.node_id, round_coord((row.geometry.x, row.geometry.y)))
    for row in toy_node.itertuples()
]

nodes = {}
node_meta = []

for node_id, coord in node_info_list:
    node_name = f"n{int(node_id)}" 
    nodes[node_name] = coord
    node_meta.append({
        "node_id": int(node_id),
        "node_name": node_name,
        "geometry": coord
    })

coord_to_node = {coord: node_name for node_name, coord in nodes.items()}

print("====== Node info (직접 매핑) ======")
for meta in node_meta:
    print(f"{meta['node_name']} (ID: {meta['node_id']}): {meta['geometry']}")



# 2. Edge numbering (e1, e2, ...)
nodeid_to_nodename = {meta["node_id"]: meta["node_name"] for meta in node_meta}
toy_edge['from_node_name'] = toy_edge['from_node_'].map(nodeid_to_nodename)
toy_edge['to_node_name'] = toy_edge['to_node_id'].map(nodeid_to_nodename)

edge_pairs = set()

for _, row in toy_edge.iterrows():
    from_node = row['from_node_name']
    to_node = row['to_node_name']
    sorted_pair = tuple(sorted([from_node, to_node]))
    edge_pairs.add(sorted_pair)

sorted_edge_pairs = sorted(edge_pairs, key=lambda x: (x[0], x[1]))
print(sorted_edge_pairs)

edges = {}
edge_number = 1

for from_node, to_node in sorted_edge_pairs:
    edges[f"e{edge_number}"] = (from_node, to_node)
    edges[f"e{edge_number + 1}"] = (to_node, from_node)
    edge_number += 2

edge_records = []
for edge_name, (from_node, to_node) in edges.items():
    match = toy_edge[((toy_edge['from_node_name'] == from_node) & (toy_edge['to_node_name'] == to_node)) |
                     ((toy_edge['from_node_name'] == to_node) & (toy_edge['to_node_name'] == from_node))]
    
    if not match.empty:
        journeys = match.iloc[0]['journeys']
        geometry = match.iloc[0]['geometry']
        edge_id = match.iloc[0]['edge_id']
    else:
        journeys = None
        geometry = None
        edge_id = None

    edge_records.append({
        'edge_id': edge_id,
        'edge_name': edge_name,
        'from_node_name': from_node,
        'to_node_name': to_node,
        'journeys': journeys,
        'geometry': geometry
    })

toy_edge_bidirectional = pd.DataFrame(edge_records)

toy_edge_bidirectional['edge_num'] = toy_edge_bidirectional['edge_name'].str.extract(r'e(\d+)').astype(int)
toy_edge_bidirectional = toy_edge_bidirectional.sort_values(by='edge_num').reset_index(drop=True)
toy_edge_bidirectional = toy_edge_bidirectional.drop(columns='edge_num')

print("\n====== Edge info ======")
pd.set_option('display.max_rows', None)
print(toy_edge_bidirectional)



# 3. Convert edges to arc format
arcs = [(u, v) for _, (u, v) in edges.items()]



# 4. Compute Euclidean distances
def euclidean_distance(node1, node2):
    x1, y1 = nodes[node1]
    x2, y2 = nodes[node2]
    return round(((x2 - x1)**2 + (y2 - y1)**2)**0.5, 2)

arc_distance = {edge_name: euclidean_distance(u, v) for edge_name, (u, v) in edges.items()}



# 5. Create a network graph
G = nx.DiGraph()

# Add nodes and edges
for node, position in nodes.items():
    G.add_node(node, pos=position)

for edge_name, (u, v) in edges.items(): 
    G.add_edge(u, v, weight=arc_distance[edge_name])

# Remove duplicate edges
unique_edges = set()
edge_name_map = {}

for edge_name, (u, v) in edges.items():
    if (v, u) not in unique_edges:
        unique_edges.add((u, v))
        
        reverse_edge_name = [k for k, (a, b) in edges.items() if (a, b) == (v, u)]
        if reverse_edge_name:
            journeys = toy_edge_bidirectional[
                (toy_edge_bidirectional['from_node_name'] == u) &
                (toy_edge_bidirectional['to_node_name'] == v)
            ]['journeys'].values

            journeys_val = int(journeys[0]) if len(journeys) > 0 else "?"
            edge_label = f"{edge_name}, {reverse_edge_name[0]} ({journeys_val})"
        else:
            edge_label = edge_name

        edge_name_map[(u, v)] = edge_label


# Plot the network topology
plt.figure(figsize=(7, 7))
pos = nx.get_node_attributes(G, 'pos')
nx.draw_networkx_nodes(G, pos, node_size=250, node_color="lightblue")
nx.draw_networkx_edges(
    G, pos,
    edgelist=list(unique_edges),  
    arrowstyle='-',
    min_target_margin=10,
    min_source_margin=10
)
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_name_map, font_size=6)
nx.draw_networkx_labels(G, pos, font_size=7)
plt.title("Network with Unique Edge Labels")
plt.show()

In [ ]:
import sys
import os
import gc
import re
import json
import copy

# Scientific Libraries
import numpy as np
from scipy.stats import beta

# Optimization
from gurobipy import Model, GRB, quicksum

# Graph & Plotting\
import matplotlib
matplotlib.use("Agg")

import networkx as nx
import matplotlib.pyplot as plt
import matplotlib as mpl

# MBNpy Modules
import matplotlib
matplotlib.use("Agg")
from mbnpy import brc, cpm, variable, operation, branch, config

# Local Module
import batch

# Clean up memory
gc.collect()
%matplotlib inline


In [ ]:
nodes = nodes  
edges = edges  
arcs = arcs
arc_distance = arc_distance 



# 1. Generate arc failure probabilities based on arc distances (longer distance -> higher failure probability)
min_dist = min(arc_distance.values())
max_dist = max(arc_distance.values())

min_prob = 0.01
max_prob = 0.3

def compute_failure_probability(distance, min_dist, max_dist, min_prob, max_prob):
    normalized_dist = (distance - min_dist) / (max_dist - min_dist)
    return round(min_prob + normalized_dist * (max_prob - min_prob), 4)

probs_dynamic = {
    edge: {
        0: compute_failure_probability(dist, min_dist, max_dist, min_prob, max_prob),
        1: round(1 - compute_failure_probability(dist, min_dist, max_dist, min_prob, max_prob), 4)
    }
    for edge, dist in arc_distance.items()
}

def numeric_sort(edge):
    return int(edge[1:])

probs_sorted = {k: probs_dynamic[k] for k in sorted(probs_dynamic, key=numeric_sort)}

probs = probs_sorted
probs_cpm = copy.deepcopy(probs)



# 2. Assign capacities to each arcs
intact_capacity = {}

for edge_name, (u, v) in edges.items():
    match = toy_edge[((toy_edge['from_node_name'] == u) & (toy_edge['to_node_name'] == v)) |
                     ((toy_edge['from_node_name'] == v) & (toy_edge['to_node_name'] == u))]

    if not match.empty:
        intact_capacity[edge_name] = match.iloc[0]['journeys']
    else:
        intact_capacity[edge_name] = None  

print("Intact Capacities:", intact_capacity)

def generate_comps_st(probs):
    comps_st = {}

    for edge, prob in probs.items():
        if isinstance(prob, dict) and 0 in prob and 1 in prob:  
            comps_st[edge] = np.random.choice([0, 1], p=[prob[0], prob[1]])
        else:
            print(f"Warning: Invalid probability format for edge {edge}: {prob}")  

    return comps_st

comps_st = generate_comps_st(probs_sorted)  



# 3. Compute maximum allowable distance
json_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\Korea\demand_data.json"
with open(json_path, "r", encoding="utf-8") as f:
    demand = json.load(f)

avg_velo = 149  # Speed in km/h
max_distance = {}
commodity_name_map = {}  

for idx, item in enumerate(demand, start=1):
    origin = item["origin"]
    destination = item["destination"]
    commodity_key = f"{origin}->{destination}"
    commodity_name = f"k{idx}"  

    distance = item["distance"]
    max_allowable_time = (distance * 60) / avg_velo + 180
    max_distance[commodity_name] = max_allowable_time * avg_velo / 60

    commodity_name_map[commodity_name] = {
        "key": commodity_key,
        "origin": origin,
        "destination": destination
    }

    print(f"\nCommodity: {commodity_name}")
    print(f"  OD Pair: {commodity_key}")
    print(f"  Distance (from JSON): {distance:.2f} km")
    print(f"  Maximum allowable time: {max_allowable_time:.2f} minutes")
    print(f"  Maximum allowable distance: {max_distance[commodity_name]:.2f} km")

In [ ]:
# Build directed graph with weighted arcs
G = nx.DiGraph()
for node, position in nodes.items():
    G.add_node(node, pos=position)

for edge_id, (u, v) in edges.items():
    G.add_edge(u, v, weight=arc_distance[edge_id])

# Initialize demand for each arc
edge_demand = { (u, v): 0 for u, v in G.edges }

# Accumulate demand from all shortest paths
for info in demand:
    origin = info["origin_name"]        
    destination = info["destination_name"]
    amount = info["journeys"]            

    try:
        paths = list(nx.all_shortest_paths(G, source=origin, target=destination, weight="weight"))
        for path in paths:
            for i in range(len(path) - 1):
                edge = (path[i], path[i + 1])
                if edge in edge_demand:
                    edge_demand[edge] += amount
                    print(f"🟦 OD {origin} → {destination}, amount={amount} ➜ edge {edge} += {amount} (total: {edge_demand[edge]})")
    except nx.NetworkXNoPath:
        print(f"No path found for {origin} → {destination}")
        continue

# Log-scaled normalization
normalized_demand = {
    (u, v): np.log10(d + 1) for (u, v), d in edge_demand.items()
}
D_max = max(normalized_demand.values())
D_min = min(normalized_demand.values())

normalized_demand = {
    (u, v): (val - D_min) / (D_max - D_min)
    for (u, v), val in normalized_demand.items()
}

# Visualization
from matplotlib.colors import LinearSegmentedColormap
start_rgb = (245/255, 250/255, 254/255)    
mid_rgb   = (100/255, 169/255, 211/255)       
end_rgb   = (8/255, 50/255, 110/255)    
cmap = LinearSegmentedColormap.from_list("custom_3color_gradient", [start_rgb, mid_rgb, end_rgb])

pos = nx.get_node_attributes(G, "pos")
fig, ax = plt.subplots(figsize=(14, 14))

edge_labels = {
    (u, v): f"{edge_demand[(u, v)]:.0f}"
    for (u, v) in G.edges
    if edge_demand[(u, v)] > 0
}

nx.draw_networkx_edge_labels(
    G,
    pos,
    edge_labels=edge_labels,
    font_size=8,
    font_color="darkred",
    rotate=False,
    ax=ax
)

for (u, v), value in normalized_demand.items():
    color = cmap(value)
    nx.draw_networkx_edges(
        G, pos, edgelist=[(u, v)],
        width=1.8,
        edge_color=[color],
        arrowstyle="->",
        arrowsize=8,
        alpha=1,
        ax=ax,
        connectionstyle="arc3,rad=0.07"
    )

ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron, crs="EPSG:3857", zorder=0)
ax.tick_params(labelbottom=True, labelleft=True)
ax.set_axis_on()
ax.ticklabel_format(useOffset=False)

# Colorbar
norm = mpl.colors.Normalize(vmin=D_min, vmax=D_max)
sm = mpl.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label("Edge Demand Intensity (normalized)")

# Grid
ax.grid(True, which='major', linestyle='--', linewidth=0.5, color='gray', alpha=0.5)
ax.tick_params(axis='both', which='major', labelsize=14, direction='out')
ax.set_axisbelow(True)  

plt.title("OD Demands for Scotland Railway Network", fontsize=18)
plt.xlabel("Longitude", fontsize=14)
plt.ylabel("Latitude", fontsize=14)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
def MCNF_od_systemfunc(arcs, comps_st, edges, arc_capacity, demand,max_distance, arc_distance, target_od):
    
    from gurobipy import Model, GRB, quicksum
    import networkx as nx

    G = nx.Graph()
    for e, (i, j) in edges.items():
        G.add_edge(i, j, weight=arc_distance.get(e, 1)) 

    model = Model("Network Flow Optimization")
    model.setParam('OutputFlag', 0) 

    # Define decision variables
    flow = {}
    unmet_demand = {}

    for k, info in demand.items():
        unmet_demand[k] = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"unsatisfied_{k}")
        for i, j in arcs:
            arc_key = next((e for e, v in edges.items() if v == (i, j) or v == (j, i)), None)
            capacity = arc_capacity.get(arc_key, 0)
            flow[k, i, j] = model.addVar(lb=0, ub=capacity, vtype=GRB.CONTINUOUS, name=f"flow_{k}_{i}_{j}")

    # Objective: minimize total unmet demand
    model.setObjective(
        quicksum(unmet_demand[k] for k in demand),
        GRB.MINIMIZE
    )

    nodes = set(node for edge in edges.values() for node in edge)

    # Constraint 1: Flow conservation
    for k, info in demand.items():
        origin = info['origin']
        destination = info['destination']
        amount = info['amount']
        for node in nodes: 
            inflow = quicksum(flow[k, i, j] for i, j in arcs if j == node)
            outflow = quicksum(flow[k, i, j] for i, j in arcs if i == node)
            if node == origin:
                model.addConstr(outflow - inflow == amount - unmet_demand[k])
            elif node == destination:
                model.addConstr(outflow - inflow == - amount + unmet_demand[k])
            else:
                model.addConstr(outflow - inflow == 0)

    # Constraint 2: Arc capacity limits
    for i, j in arcs:
        arc_key = next((e for e, v in edges.items() if v == (i, j) or v == (j, i)), None)
        model.addConstr(quicksum(flow[k, i, j] for k in demand if (k, i, j) in flow) <= arc_capacity.get(arc_key, 0))

    # Constraint 3: Distance constraints for each commodity
    for k, info in demand.items():
        origin = info['origin']
        distance_expr = quicksum(arc_distance.get(e, 0) * flow[k, i, j] for e, (i, j) in edges.items() if (k, i, j) in flow)
        total_flow = quicksum(flow[k, i, j] for i, j in arcs if (k, i, j) in flow and i == origin)
        
        model.addConstr(distance_expr <= max_distance[k] * total_flow)

    model.optimize()
    
    if model.status == GRB.OPTIMAL:
        origin = demand[target_od]['origin']
        destination = demand[target_od]['destination']
        amount = demand[target_od]['amount']

        # Sum of flow into destination
        flow_to_dest = sum(
            flow[target_od, i, destination].x
            for i, j in arcs
            if j == destination and (target_od, i, j) in flow
        )

        flow_ratio = flow_to_dest / amount if amount > 0 else 0
        if flow_ratio > 0.9:
            sys_st = 's'

            min_comps_st = {}
            for (k, i, j), var in flow.items():
                if k == target_od and var.x > 0:
                    link_name = next((e for e, v in edges.items() if v == (i, j) or v == (j, i)), None)
                    if link_name:
                        min_comps_st[link_name] = 1

        else:
            sys_st = 'f'
            min_comps_st = None    
        
        print(f"Used Components: {min_comps_st}")

        return flow_ratio, sys_st, min_comps_st
    
    else:
        return None, None, None

In [ ]:
def MCNF_systemfunc(arcs, comps_st, edges, arc_capacity, demand, max_distance, arc_distance):
    from gurobipy import Model, GRB, quicksum
    import networkx as nx

    G = nx.Graph()
    for e, (i, j) in edges.items():
        G.add_edge(i, j, weight=arc_distance.get(e, 1))  

    model = Model("Network Flow Optimization")
    model.setParam('OutputFlag', 0) 

    # Define variables
    flow = {}
    unmet_demand = {}

    for k, info in demand.items():
        unmet_demand[k] = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"unsatisfied_{k}")
        for i, j in arcs:
            arc_key = next((e for e, v in edges.items() if v == (i, j) or v == (j, i)), None)
            capacity = arc_capacity.get(arc_key, 0)
            flow[k, i, j] = model.addVar(lb=0, ub=capacity, vtype=GRB.CONTINUOUS, name=f"flow_{k}_{i}_{j}")

    # Objective function: Minimize expected loss
    model.setObjective(
        quicksum(unmet_demand[k] for k in demand),
        GRB.MINIMIZE
    )

    # Extract all nodes from edge values
    nodes = set(node for edge in edges.values() for node in edge)

    # Constraint 1: Flow conservation
    for k, info in demand.items():
        origin = info['origin']
        destination = info['destination']
        amount = info['amount']
        for node in nodes: 
            inflow = quicksum(flow[k, i, j] for i, j in arcs if j == node)
            outflow = quicksum(flow[k, i, j] for i, j in arcs if i == node)
            if node == origin:
                model.addConstr(outflow - inflow == amount - unmet_demand[k])
            elif node == destination:
                model.addConstr(outflow - inflow == - amount + unmet_demand[k])
            else:
                model.addConstr(outflow - inflow == 0)

    # Constraint 2: Arc capacity limits
    for i, j in arcs:
        arc_key = next((e for e, v in edges.items() if v == (i, j) or v == (j, i)), None)
        model.addConstr(quicksum(flow[k, i, j] for k in demand if (k, i, j) in flow) <= arc_capacity.get(arc_key, 0))

    # Constraint 3: Commodity별 max_distance 적용
    for k, info in demand.items():
        origin = info['origin']
        distance_expr = quicksum(arc_distance.get(e, 0) * flow[k, i, j] for e, (i, j) in edges.items() if (k, i, j) in flow)
        total_flow = quicksum(flow[k, i, j] for i, j in arcs if (k, i, j) in flow and i == origin)
        
        model.addConstr(distance_expr <= max_distance[k] * total_flow)

    # Perform optimization
    model.optimize()

    # Process results
    if model.status == GRB.OPTIMAL:
        expected_loss = model.objVal

        total_demand = sum(info['amount'] for info in demand.values())
        expected_loss = max(0.0, min(expected_loss, total_demand)) 

        if expected_loss < 127364:
            sys_st = 's'

            min_comps_st = {}

            for (k, i, j), var in flow.items():
                if var.x > 0:
                    link_name = next((e for e, v in edges.items() if v == (i, j) or v == (j, i)), None)
                    if link_name:
                        min_comps_st[link_name] = 1 

        else:
            sys_st = 'f'
            min_comps_st = None
        
        return expected_loss, sys_st, min_comps_st

    else:
        return None, None, None  

In [ ]:
import networkx as nx
from networkx.algorithms.flow import shortest_augmenting_path

def shortestpath_systemfunc(arcs, comps_st, edges, arc_capacity, demand, max_distance, arc_distance):

    G = nx.DiGraph()

    for e, (i, j) in edges.items():
        if comps_st.get(e, 0) > 0:
            cap = arc_capacity.get(e, 1)
            dist = arc_distance.get(e, 1)
            G.add_edge(i, j, capacity=cap, weight=dist, link_id=e)
            G.add_edge(j, i, capacity=cap, weight=dist, link_id=e)

    expected_loss = 0.0
    used_links_set = set()

    for k, info in demand.items():
        origin = info["origin"]
        destination = info["destination"]
        amount = info["amount"]
        max_dist = max_distance.get(k, float('inf'))

        if origin not in G.nodes or destination not in G.nodes:
            expected_loss += amount
            continue

        G_temp = G.copy()
        G_temp.add_edge(destination, 'sink', capacity=1) 

        try:
            flow_value, flow_dict = nx.maximum_flow(
                G_temp, origin, 'sink', capacity='capacity', flow_func=shortest_augmenting_path
            )
        except Exception as e:
            flow_value = 0

        if flow_value == 0:
            expected_loss += amount
            continue

        try:
            path = nx.shortest_path(G, source=origin, target=destination, weight='weight')
            path_length = nx.shortest_path_length(G, source=origin, target=destination, weight='weight')

            if path_length > max_dist:
                expected_loss += amount
            else:
                for u, v in zip(path[:-1], path[1:]):
                    edge_id = G[u][v]['link_id']
                    used_links_set.add(edge_id)
        except nx.NetworkXNoPath:
            expected_loss += amount

    total_demand = sum(info["amount"] for info in demand.values())
    expected_loss = max(0.0, min(expected_loss, total_demand))

    if expected_loss < 127364:
        sys_st = 's'
        min_comps_st = {e: 1 for e in used_links_set}
    else:
        sys_st = 'f'
        min_comps_st = None

    return expected_loss, sys_st, total_demand


모든 컴포넌트가 정상 상태일 때 expected loss

In [ ]:
demand_dict = {}

for idx, item in enumerate(demand, start=1):
    key = f"k{idx}"
    demand_dict[key] = {
        "origin": item["origin_name"],
        "destination": item["destination_name"],
        "amount": item["journeys"],
        "distance": item["distance"]
    }

comps_st = {edge: 1 for edge in intact_capacity}
arc_capacity = {edge: intact_capacity[edge] * comps_st[edge] for edge in intact_capacity}

expected_loss, sys_st, total_demand = shortestpath_systemfunc(
    arcs=arcs,
    comps_st=comps_st,
    edges=edges,
    arc_capacity=arc_capacity,
    demand=demand_dict,
    max_distance=max_distance,
    arc_distance=arc_distance
)

print("Expected Loss:", expected_loss)
print("System State:", sys_st)
print("Total Demand:", total_demand)

Run MCNF system function

In [ ]:
sys_fun_mcnf = lambda comps_st: MCNF_systemfunc(
    arcs=arcs,
    comps_st=comps_st,
    edges=edges,
    arc_capacity={e: intact_capacity[e] * comps_st[e] for e in comps_st},
    demand=demand_dict,
    max_distance=max_distance,
    arc_distance=arc_distance
)

sys_fun_shortest = lambda comps_st: shortestpath_systemfunc(
    arcs=arcs,
    comps_st=comps_st,
    edges=edges,
    arc_capacity={e: intact_capacity[e] * comps_st[e] for e in comps_st},
    demand=demand_dict,
    max_distance=max_distance,
    arc_distance=arc_distance
)

print("\n🔹 Input Values:")
print("Component States (comps_st):", comps_st)
print("Edges:", edges)
print("Arc Capacity:", arc_capacity)
print("Demand:", demand_dict)
print("Max Distance:", max_distance)
print("Arc Distance:", arc_distance)

expected_loss_mcnf, sys_st_mcnf, min_comps_st_mcnf = sys_fun_mcnf(comps_st)
expected_loss_shortest, sys_st_shortest, min_comps_st_shortest = sys_fun_shortest(comps_st)

print("\n🔹 Output Values:")
print("System State (MCNF):", sys_st_mcnf)
print("System State (Shortest):", sys_st_shortest)
print("Minimum component state (MCNF):", min_comps_st_mcnf) 
print("Minimum component state (Shortest):", min_comps_st_shortest) 

# Expected Loss Evaluation
By BRC algorithm

### Shortest path

In [ ]:
brs_shortest, rules_shortest, sys_res_shortest, monitor_shortest = brc.run(
    probs=probs, 
    sys_fun=sys_fun_shortest, 
    max_sf=np.inf, 
    max_nb=np.inf, 
    pf_bnd_wr=0.0, 
    max_rules=100,  
    active_decomp=10,
    display_freq=5,
    brs=[],
)

# Check if BRC stopped due to max_rules
if len(rules_shortest['s'] + rules_shortest['f']) >= 100:
    print("\n🔹 BRC terminated because 100 rules have been found.")

# System failure probability
Pf_shortest = sum(branch.p for branch in brs_shortest if branch.down_state == 'f')
print(f"System Failure Probability (P_f): {Pf_shortest}")

# Identify unknown branches (brs_u)
brs_u_shortest = [branch for branch in brs_shortest if branch.down_state == 'u' or branch.up_state == 'u']

# Print rules
print("All Survival Rules:")
for i, rule in enumerate(rules_shortest['s'], 1):
    print(f"Rule {i}: {rule}")

print("\nAll Failure Rules:")
for i, rule in enumerate(rules_shortest['f'], 1):
    print(f"Rule {i}: {rule}")

In [ ]:
# Print branches
print("MCNF All Branches:")
for i, branch in enumerate(brs_shortest, 1):
    print(f"Branch {i}:")
    print(f"  Down: {branch.down}")
    print(f"  Up: {branch.up}")
    print(f"  Down State: {branch.down_state}")
    print(f"  Up State: {branch.up_state}")
    print(f"  Probability: {branch.p}\n")

In [ ]:
print("\n🔹 Running Monte Carlo Simulation (MCS) after BRC termination.")

def sys_fun_rs_shortest(sample1):
    val, sys_st, _ = sys_fun_shortest(sample1)
    return val, sys_st

# Define system variable
varis = {}
for k in edges:
    varis[k] = variable.Variable(name=k, values=[0, 1]) 
varis['sys_event'] = variable.Variable(name='sys_event', values=['f', 's', 'u'])

# Create CPMs for each component event
cpms = {}  # Initialize CPMs
for k, v in edges.items():
    cpms[k] = cpm.Cpm(
        variables=[varis[k]], 
        no_child=1, 
        C=np.array([[0], [1]]),  # Define states (0: fail, 1: survive)
        p=np.array([probs[k][0], probs[k][1]])  # Assign probabilities
    )

def get_composite_state(vari, states):
    added = set(states)
    if added not in vari.B:
        vari.B.append(added)
    cst = vari.B.index(added)
    return vari, cst

import mbnpy.variable as variable
variable.get_composite_state = get_composite_state

# Generate system constraint matrix
Csys, varis = brc.get_csys(brs_shortest, varis, {'f': 0, 's': 1, 'u': 2})
Csys = np.array(Csys, dtype=np.int32)  # Ensure integer type

# Ensure "sys_event" exists in CPMs
if "sys_event" not in cpms:
    cpms["sys_event"] = cpm.Cpm(
        variables=[varis["sys_event"]] + [varis[e] for e in edges],
        no_child=1,
        C=np.zeros((1, len(edges) + 1), dtype=int),  # Initialize constraints matrix
        p=np.array([1.0])  # Set initial probability
    )

# Adjust constraint matrix dimensions if needed
expected_columns = len(cpms["sys_event"].variables)
actual_columns = Csys.shape[1]

if actual_columns != expected_columns:
    print(f"⚠ Warning: Adjusting Csys! Expected {expected_columns} columns, but got {actual_columns}.")
    if actual_columns > expected_columns:
        Csys = Csys[:, :expected_columns]  # Trim excess columns
    else:
        missing_cols = expected_columns - actual_columns
        extra_cols = np.full((Csys.shape[0], missing_cols), 0)  # Fill missing columns with 0
        Csys = np.hstack((Csys, extra_cols))

cpms["sys_event"].Cs = Csys  # Assign constraint matrix to CPMs


# Transform unknown branches for MCS
brs_u_transformed_shortest = [
    (
        b.down,
        b.up,
        round(float(b.p), 20),  # Convert probability to float and round
        b.down_state,
        b.up_state,
    )
    for b in brs_u_shortest
]

# Run Monte Carlo Simulation for unknown branches
cpms_shortest, mcs_result_shortest = batch.mcs_unknown(
    brs_u=brs_u_transformed_shortest,
    probs=probs,
    sys_fun_rs=sys_fun_rs_shortest,
    cpms=cpms,
    sys_name="sys_event",
    cov_t=0.1,  # Convergence threshold
    sys_st_monitor=0,  # System failure monitoring state
    sys_st_prob=round(Pf_shortest, 20),  # Round system failure probability
    rand_seed=1  # Set random seed for reproducibility
)

# Print MCS results
print(f"\n🔹 MCS Completed. Failure Probability (pf): {mcs_result_shortest['pf']:.10e}")
print(f"   - COV: {mcs_result_shortest['cov']:.4e}")
print(f"   - Confidence Interval: [{mcs_result_shortest['cint_low']:.10e}, {mcs_result_shortest['cint_up']:.10e}]")
print(f"   - Number of Samples: {mcs_result_shortest['nsamp']}")

In [ ]:
import importlib
import batch

importlib.reload(batch)

def sys_fun_rs_shortest(sample):
    val, sys_st, _ = sys_fun_shortest(sample)
    return val, sys_st

In [ ]:
# Dictionary to store final results for each component
final_results_1 = {}

# Iterate over all components in probs_cpm
for X_n in probs_cpm.keys():  
    print(f"... Computing for Component: {X_n}")

    # Step 1: Filter branches and apply system function (using P(X_n=1))
    survival_known_branch1_shortest, unknown_branch1_shortest = batch.eventspace_x1_filter(brs_u_shortest, X_n, probs_cpm[X_n][1], sys_fun_shortest)

    # Step 2: Compute probabilities
    total_prob_survival1_shortest = batch.compute_total_probability(survival_known_branch1_shortest)
    total_prob_unknown1_shortest = batch.compute_total_probability(unknown_branch1_shortest)
    P_Xi_1 = probs_cpm[X_n][1]  # Extract P(X_i=1) from probs_cpm

    # Step 3: Compute P(S=1, X_i=1) from known branches
    B_s_shortest = [branch for branch in brs_shortest if branch.down_state == 's']
    brc_survival_prob1_shortest = batch.survivalprob_xi1_brc100(brc_branches=B_s_shortest, probs=probs_cpm, target_xi=X_n)

    # Step 4: Run Monte Carlo Simulation (MCS) for unknown branches
    survival_prob1_shortest = brc_survival_prob1_shortest + total_prob_survival1_shortest

    mcs_result_unknown_shortest = batch.run_mcs_for_unknown_branch(
        brs_u=brs_u_shortest,
        unknown_branch=unknown_branch1_shortest, 
        probs=probs_cpm,  
        sys_fun_rs=sys_fun_rs_shortest, 
        cov_t=0.01,  
        sys_st_monitor=1,
        survival_prob=survival_prob1_shortest,
        rand_seed=None
    )

    # Compute conditional probability P(S=1 | X_i=1)
    P_S_given_Xi_1_shortest = mcs_result_unknown_shortest['ps'] / P_Xi_1
    P_S_given_Xi_1_low_shortest = mcs_result_unknown_shortest['cint_low'] / P_Xi_1
    P_S_given_Xi_1_up_shortest = mcs_result_unknown_shortest['cint_up'] / P_Xi_1
    COV1 = mcs_result_unknown_shortest['cov']

    # Compute alpha and beta for Beta distribution
    alpha1, beta1 = batch.beta_parameters(P_S_given_Xi_1_shortest, COV1)

    # Store results
    final_results_1[X_n] = {
        'P(X_i=1, S=1)': mcs_result_unknown_shortest['ps'],
        'P(S=1 | X_i=1)': P_S_given_Xi_1_shortest,
        'COV': COV1,
        'Confidence Interval': [P_S_given_Xi_1_low_shortest, P_S_given_Xi_1_up_shortest],
        'Number of Samples': mcs_result_unknown_shortest['nsamp'],
        'Alpha': alpha1,
        'Beta': beta1
    }

# Print final results for all components
print("\n🔹 **Final Conditional Probabilities for All Components** 🔹")
for comp, values in final_results_1.items():
    print(f"\nComponent: {comp}")
    print(f"   - Estimated P(S=1 | {comp}=1): {values['P(S=1 | X_i=1)']:.10e}")
    print(f"   - COV: {values['COV']:.4e}")
    print(f"   - Confidence Interval: [{values['Confidence Interval'][0]:.10e}, {values['Confidence Interval'][1]:.10e}]")
    print(f"   - Number of Samples: {values['Number of Samples']}")
    print(f"   - Alpha: {values['Alpha']:.4e}, Beta: {values['Beta']:.4e}")

In [ ]:
# Dictionary to store final results for each component
final_results_0 = {}

# Iterate over all components in probs_cpm
for X_n in probs_cpm.keys():  
    print(f"... Computing for Component: {X_n}")

    # Step 1: Filter branches and apply system function
    survival_known_branch_shortest, unknown_branch_shortest = batch.eventspace_x0_filter(brs_u_shortest, X_n, probs_cpm[X_n][0], sys_fun_shortest)

    # Step 2: Compute probabilities
    total_prob_survival_shortest = batch.compute_total_probability(survival_known_branch_shortest)
    total_prob_unknown_shortest = batch.compute_total_probability(unknown_branch_shortest)
    P_Xi_0_shortest = probs_cpm[X_n][0]  # Extract P(X_i=0) from probs_cpm

    # Step 3: Compute P(S=1, X_i=0) from known branches
    B_s_shortest = [branch for branch in brs_shortest if branch.down_state == 's']
    brc_survival_prob_shortest = batch.survivalprob_xi0_brc100(brc_branches=B_s_shortest, probs=probs_cpm, target_xi=X_n)

    # Step 4: Run Monte Carlo Simulation (MCS) for unknown branches
    survival_prob = brc_survival_prob_shortest + total_prob_survival_shortest

    mcs_result_unknown_0 = batch.run_mcs_for_unknown_branch(
        brs_u=brs_u_shortest,
        unknown_branch=unknown_branch_shortest,
        probs=probs_cpm,  
        sys_fun_rs=sys_fun_rs_shortest, 
        cov_t=0.01,  
        sys_st_monitor=1,
        survival_prob=survival_prob,
        rand_seed=None
    )

    # Compute conditional probability P(S=1 | X_i=0)
    P_S_given_Xi_0_shortest = mcs_result_unknown_0['ps'] / P_Xi_0_shortest
    P_S_given_Xi_0_low_shortest = mcs_result_unknown_0['cint_low'] / P_Xi_0_shortest
    P_S_given_Xi_0_up_shortest = mcs_result_unknown_0['cint_up'] / P_Xi_0_shortest
    COV0 = mcs_result_unknown_0['cov']

    # Compute alpha and beta for Beta distribution
    alpha0, beta0 = batch.beta_parameters(P_S_given_Xi_0_shortest, COV0)

    # Store results
    final_results_0[X_n] = {
        'P(X_i=0, S=1)': mcs_result_unknown_0['ps'],
        'P(S=1 | X_i=0)': P_S_given_Xi_0_shortest,
        'COV': COV0,
        'Confidence Interval': [P_S_given_Xi_0_low_shortest, P_S_given_Xi_0_up_shortest],
        'Number of Samples': mcs_result_unknown_0['nsamp'],
        'Alpha': alpha0,
        'Beta': beta0
    }

# Print final results for all components
print("\n🔹 **Final Conditional Probabilities for All Components** 🔹")
for comp, values in final_results_0.items():
    print(f"\nComponent: {comp}")
    print(f"   - Estimated P(S=1 | {comp}=0): {values['P(S=1 | X_i=0)']:.10e}")
    print(f"   - COV: {values['COV']:.4e}")
    print(f"   - Confidence Interval: [{values['Confidence Interval'][0]:.10e}, {values['Confidence Interval'][1]:.10e}]")
    print(f"   - Number of Samples: {values['Number of Samples']}")
    print(f"   - Alpha: {values['Alpha']:.4e}, Beta: {values['Beta']:.4e}")

In [ ]:
x_values = np.linspace(0, 1, 100)   # Define the range of x values for Beta PDF
num_components = len(probs_cpm.keys())  # Number of components

# Automatically determine grid size (rows and columns)
ncols = min(6, num_components)  
nrows = (num_components + ncols - 1) // ncols 

# Create subplots
fig, axes = plt.subplots(nrows, ncols, figsize=(5 * ncols, 4 * nrows))
axes = np.array(axes).flatten()  

for ax, X_n in zip(axes, probs_cpm.keys()):
    print(f"Computing Beta PDF for Component: {X_n}")

    alpha1 = final_results_1[X_n]['Alpha']
    beta1 = final_results_1[X_n]['Beta']
    alpha2 = final_results_0[X_n]['Alpha']
    beta2 = final_results_0[X_n]['Beta']

    # Compute Beta PDFs
    beta_pdf_1 = beta.pdf(x_values, alpha1, beta1)
    beta_pdf_2 = beta.pdf(x_values, alpha2, beta2)
   
    ax.plot(x_values, beta_pdf_1, label=f"P(S=1 | {X_n}=1)", color='blue')  # Plot Beta PDF for P(S=1 | X_i=1)
    ax.plot(x_values, beta_pdf_2, label=f"P(S=1 | {X_n}=0)", color='red')   # Plot Beta PDF for P(S=1 | X_i=0)
    ax.set_title(f"Component {X_n} Beta PDF")
    ax.set_xlabel("Probability")
    ax.set_ylabel("Density")
    ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

y_values = np.linspace(-1, 1, 100)  # Define Y values for BM distribution
num_components = len(probs_cpm.keys())  # Number of components

# Automatically determine grid size (rows and columns)
ncols = min(6, num_components) 
nrows = (num_components + ncols - 1) // ncols 

# Create subplots
fig, axes = plt.subplots(nrows, ncols, figsize=(5 * ncols, 4 * nrows))
axes = np.array(axes).flatten() 

bm_results = {}
bm_confidence_intervals = {}
most_probable_bm_values = [] 
component_names = []        

for ax, X_n in zip(axes, probs_cpm.keys()):
    print(f"Computing BM distribution with mpmath for Component: {X_n}")

    alpha1 = final_results_1[X_n]['Alpha']
    beta1 = final_results_1[X_n]['Beta']
    alpha2 = final_results_0[X_n]['Alpha']
    beta2 = final_results_0[X_n]['Beta']
    
    # Compute BM distribution
    BM_distribution = np.array([
        batch.f_Y_mpmath(y, alpha1, beta1, alpha2, beta2) for y in y_values
    ])

    # Check for invalid values
    if np.sum(BM_distribution) == 0 or not np.all(np.isfinite(BM_distribution)):
        print(f"❌ Skipping component: {X_n} due to invalid BM values.")
        for y, val in zip(y_values, BM_distribution):
            if not np.isfinite(val):
                print(f"   → Y = {y:.4f}, f_Y = {val}")
        continue

    # Mode = Y with max f_Y
    mode_BM = y_values[np.argmax(BM_distribution)]

    # Store results
    bm_results[X_n] = BM_distribution
    bm_confidence_intervals[X_n] = {
        'Lower Bound': y_values[np.searchsorted(np.cumsum(BM_distribution) / np.sum(BM_distribution), 0.025)],
        'Upper Bound': y_values[np.searchsorted(np.cumsum(BM_distribution) / np.sum(BM_distribution), 0.975)],
        'Most Probable BM': mode_BM
    }

    # 🔸 추가: bar chart용 값 저장
    component_names.append(X_n)
    most_probable_bm_values.append(mode_BM)

    # Plot
    ax.plot(y_values, BM_distribution, label=f"{X_n}", color='blue')
    ax.axvline(mode_BM, color='purple', linestyle='-', label="Most Probable BM")
    ax.set_title(f"Component {X_n}")
    ax.set_xlabel("Y = P(S=1 | X_i=1) - P(S=1 | X_i=0)")
    ax.set_ylabel("Density")
    ax.legend()

plt.tight_layout()
plt.show()

# 🔸 Bar Chart 바로 이어서
plt.figure(figsize=(8, 5))
plt.bar(component_names, most_probable_bm_values, color='gray', alpha=0.7)
plt.xlabel("Component")
plt.ylabel("Most Probable BM Value")
plt.title("Component Importance Based on Birnbaum's Measure (BRC+Sampling)")
plt.ylim(-0.05, 1) 
plt.xticks(rotation=60, fontsize=6) 
plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.show()


### MCNF

In [ ]:
# Run BRC with stopping criteria (max_rules = 100)
brs_mcnf, rules_mcnf, sys_res_mcnf, monitor_mcnf = brc.run(
    probs=probs, 
    sys_fun=sys_fun_mcnf, 
    max_sf=np.inf, 
    max_nb=np.inf, 
    pf_bnd_wr=0.0, 
    max_rules=100,  
    active_decomp=10,
    display_freq=5,
    brs=[],
)

# Check if BRC stopped due to max_rules
if len(rules_mcnf['s'] + rules_mcnf['f']) >= 100:
    print("\n🔹 BRC terminated because 100 rules have been found.")

# System failure probability
Pf_mcnf = sum(branch.p for branch in brs_mcnf if branch.down_state == 'f')
print(f"System Failure Probability (P_f): {Pf_mcnf}")

# Identify unknown branches (brs_u)
brs_u_mcnf = [branch for branch in brs_mcnf if branch.down_state == 'u' or branch.up_state == 'u']

# Print survival rules
print("All Survival Rules:")
for i, rule in enumerate(rules_mcnf['s'], 1):
    print(f"Rule {i}: {rule}")

# Print failure rules
print("\nAll Failure Rules:")
for i, rule in enumerate(rules_mcnf['f'], 1):
    print(f"Rule {i}: {rule}")

In [ ]:
# Print branches
print("MCNF All Branches:")
for i, branch in enumerate(brs_mcnf, 1):
    print(f"Branch {i}:")
    print(f"  Down: {branch.down}")
    print(f"  Up: {branch.up}")
    print(f"  Down State: {branch.down_state}")
    print(f"  Up State: {branch.up_state}")
    print(f"  Probability: {branch.p}\n")

In [ ]:
print("\n🔹 Running Monte Carlo Simulation (MCS) after BRC termination.")

def sys_fun_rs(sample1):
    val, sys_st, _ = sys_fun_mcnf(sample1)
    return val, sys_st

# Define system variable
varis = {}
for k in edges:
    varis[k] = variable.Variable(name=k, values=[0, 1]) 
varis['sys_event'] = variable.Variable(name='sys_event', values=['f', 's', 'u'])

# Create CPMs for each component event
cpms = {}  # Initialize CPMs
for k, v in edges.items():
    cpms[k] = cpm.Cpm(
        variables=[varis[k]], 
        no_child=1, 
        C=np.array([[0], [1]]),  # Define states (0: fail, 1: survive)
        p=np.array([probs[k][0], probs[k][1]])  # Assign probabilities
    )

def get_composite_state(vari, states):
    added = set(states)
    if added not in vari.B:
        vari.B.append(added)
    cst = vari.B.index(added)
    return vari, cst

import mbnpy.variable as variable
variable.get_composite_state = get_composite_state

# Generate system constraint matrix
Csys, varis = brc.get_csys(brs_mcnf, varis, {'f': 0, 's': 1, 'u': 2})
Csys = np.array(Csys, dtype=np.int32)  # Ensure integer type

# Ensure "sys_event" exists in CPMs
if "sys_event" not in cpms:
    cpms["sys_event"] = cpm.Cpm(
        variables=[varis["sys_event"]] + [varis[e] for e in edges],
        no_child=1,
        C=np.zeros((1, len(edges) + 1), dtype=int),  # Initialize constraints matrix
        p=np.array([1.0])  # Set initial probability
    )

# Adjust constraint matrix dimensions if needed
expected_columns = len(cpms["sys_event"].variables)
actual_columns = Csys.shape[1]

if actual_columns != expected_columns:
    print(f"⚠ Warning: Adjusting Csys! Expected {expected_columns} columns, but got {actual_columns}.")
    if actual_columns > expected_columns:
        Csys = Csys[:, :expected_columns]  # Trim excess columns
    else:
        missing_cols = expected_columns - actual_columns
        extra_cols = np.full((Csys.shape[0], missing_cols), 0)  # Fill missing columns with 0
        Csys = np.hstack((Csys, extra_cols))

cpms["sys_event"].Cs = Csys  # Assign constraint matrix to CPMs


# Transform unknown branches for MCS
brs_u_transformed_mcnf = [
    (
        b.down,
        b.up,
        round(float(b.p), 20),  # Convert probability to float and round
        b.down_state,
        b.up_state,
    )
    for b in brs_u_mcnf
]

# Run Monte Carlo Simulation for unknown branches
cpms_mcnf, mcs_result_mcnf = batch.mcs_unknown(
    brs_u=brs_u_transformed_mcnf,
    probs=probs,
    sys_fun_rs=sys_fun_rs,
    cpms=cpms,
    sys_name="sys_event",
    cov_t=0.01,  # Convergence threshold
    sys_st_monitor=0,  # System failure monitoring state
    sys_st_prob=round(Pf_mcnf, 20),  # Round system failure probability
    rand_seed=1  # Set random seed for reproducibility
)

# Print MCS results
print(f"\n🔹 MCS Completed. Failure Probability (pf): {mcs_result_mcnf['pf']:.10e}")
print(f"   - COV: {mcs_result_mcnf['cov']:.4e}")
print(f"   - Confidence Interval: [{mcs_result_mcnf['cint_low']:.10e}, {mcs_result_mcnf['cint_up']:.10e}]")
print(f"   - Number of Samples: {mcs_result_mcnf['nsamp']}")

In [ ]:
#### **OD마다 BRC 돌리는 코드**

import numpy as np

##### 결과 저장 딕셔너리
brc_results_by_od = {}

demand_dict = {}

for idx, item in enumerate(demand, start=1):
    key = f"k{idx}"
    demand_dict[key] = {
        "origin": item["origin_name"],
        "destination": item["destination_name"],
        "amount": item["journeys"],
        "distance": item["distance"]
    }
    
##### 각 OD에 대해 반복
for od_key in demand_dict:
    print(f"\n🔍 Running BRC for OD pair: {od_key} ...")

    sys_fun = lambda comps_st, od=od_key: MCNF_od_systemfunc(
    arcs=arcs, 
    comps_st=comps_st, 
    edges=edges, 
    arc_capacity={e: int(intact_capacity[e] * comps_st[e]) for e in comps_st},
    demand=demand_dict,
    max_distance=max_distance, 
    arc_distance=arc_distance, 
    target_od=od)

    # BRC 실행
    brs, rules, sys_res, monitor = brc.run(
        probs=probs,
        sys_fun=sys_fun,
        max_sf=np.inf,
        max_nb=np.inf,
        pf_bnd_wr=0.0,
        max_rules=30,
        active_decomp=10,
        display_freq=5,
        brs=[],
    )

    # 결과 정리
    P_f = sum(branch.p for branch in brs if branch.down_state == 'f')

    print(f"→ OD {od_key}: P_f = {P_f:.6f}")
    if len(rules['s'] + rules['f']) >= 100:
        print("⚠️  BRC stopped due to max_rules=100")

    # 저장
    brc_results_by_od[od_key] = {
        'brs': brs,
        'rules': rules,
        'P_f': P_f,
        'sys_res': sys_res,
        'monitor': monitor,
    }

    print(rules)


In [ ]:
import json

def serialize_brc_results(results_dict):
    serializable_results = {}

    for od_key, result in results_dict.items():
        serializable_results[od_key] = {
            'P_f': result['P_f'],
            'rule_count': {
                's': len(result['rules'].get('s', [])),
                'f': len(result['rules'].get('f', [])),
            },
            'rules': {
                's': [rule['f'] for rule in result['rules'].get('s', [])],
                'f': [rule['f'] for rule in result['rules'].get('f', [])],
            }
        }
    return serializable_results

serializable_data = serialize_brc_results(brc_results_by_od)

output_path = "brc_results_by_od_summary.json"
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(serializable_data, f, indent=2, ensure_ascii=False)

print(f"Saved summary to {output_path}")